In [ ]:
purpose = "driving organic traffic"
target_audience = "drupal users"
pain_points = []

primary_keywords = []
secondary_keywords = []

search_intent = ""
competitors_urls = [
"https://medium.com/valuebound/ultimate-guide-to-drupal-for-beginners-9261e4b31160",
"https://www.valuebound.com/blog/boosting-your-digital-presence-drupal",
"https://www.acquia.com/blog/drupal-11"
]

topic = ""

In [ ]:
# !pip install numpy rank-bm25 chromadb 
# !pip install langchain langchain-community sentence-transformers


In [ ]:
# !pip install beautifulsoup4


In [ ]:
!pip install langchain-openai


In [ ]:
from langchain_openai import AzureChatOpenAI

import os

import time
AZURE_OPENAI_API_KEY=""
AZURE_OPENAI_ENDPOINT=""
OPENAI_MODEL_VERSION=""


model = AzureChatOpenAI(
    azure_deployment="gpt-4o-mini",
    api_version=OPENAI_MODEL_VERSION,
    temperature=0.75,
    max_tokens=2056,
    max_retries=100,
    api_key=AZURE_OPENAI_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT
)

def send_request_to_llm(formatted_prompt) -> str:
    retries = 3

    for attempt in range(retries):
        try:
            print("calling LLM", AZURE_OPENAI_API_KEY, AZURE_OPENAI_ENDPOINT, OPENAI_MODEL_VERSION)
            # Increment the LLM call counter in Redis

            res = model.invoke(formatted_prompt)
            print(type(res), res.content)
            return res.content

        except Exception as e:
            print(f"Error while calling LLM on attempt {attempt + 1}: {e}")
            if attempt < retries - 1:
                print("Retrying in 3 seconds...")
                time.sleep(3)

    print("LLM call failed after 3 retries")
    return "LLM call failed"


send_request_to_llm("hi")

calling LLM 1PWWT1ehinTCU2KNZE7OuE6o8ELkCtRufSrOqOvGxk48lTHZ6ZfCJQQJ99BAACLArgHXJ3w3AAAAACOGsb6z https://ai-tmhub2080459798235.openai.azure.com/ 2024-05-01-preview
<class 'langchain_core.messages.ai.AIMessage'> Hello! How can I assist you today?


'Hello! How can I assist you today?'

In [5]:
import requests
from bs4 import BeautifulSoup
import os
def fetch_webpage(url):
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()  # Raise an error for bad responses (4xx and 5xx)
        
        soup = BeautifulSoup(response.text, 'html.parser')
        page_content = soup.prettify()  # Stores the entire parsed HTML content
        
        return page_content
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the webpage: {e}")
        return None


# for i,url in enumerate(competitors_urls):
#   webpage_content = fetch_webpage("https://r.jina.ai//"+url)
#   #write webpage content to a file
#   os.makedirs("blogs", exist_ok=True)
#   with open(f"blogs/{i}.md", "w", encoding="utf-8") as file:
#     file.write(webpage_content)


In [6]:
## load all .md files from blogs folder 
def read_file_content(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
        return content
    except FileNotFoundError:
        return "Error: File not found."
    except Exception as e:
        return f"Error: {e}"

def get_markdown_files(folder_path):
    try:
        md_files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith(".md")]
        return md_files
    except FileNotFoundError:
        return "Error: Folder not found."
    except Exception as e:
        return f"Error: {e}"


In [ ]:
get_markdown_files("blogs")


In [7]:
from chroma_search import ChromaSearch, get_chroma_hybrid_search
from langchain.text_splitter import RecursiveCharacterTextSplitter

def embed(path: str, chroma_search: ChromaSearch):
    with open(path, 'r', encoding='utf-8') as f:
        file = f.read()

    name = path.split("\\")[-1]

    # Split text into chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    chunks = text_splitter.split_text(file)

    for i, chunk in enumerate(chunks):
        chunk_name = f"{name}_part_{i}"
        chroma_search.upload_document(chunk, chunk_name, "all", chunk_name)

d:\Projects\content_generation\chroma_search.py:10: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L12-v2")
d:\Projects\content_generation\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
import re

def clean_md(file_path):
    """
    Cleans a markdown file by removing image links, hyperlinks, and embedded links.
    It overwrites the file with the cleaned content.
    
    :param file_path: Path to the markdown file.
    """
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    
    # Remove image links ![alt text](image_url)
    content = re.sub(r'!\[.*?\]\(.*?\)', '', content)
    
    # Remove hyperlinks [text](url)
    content = re.sub(r'\[([^\]]+)\]\((.*?)\)', r'\1', content)
    
    # Remove raw URLs
    content = re.sub(r'http[s]?://\S+', '', content)
    
    # Remove excessive empty lines
    content = re.sub(r'\n{2,}', '\n\n', content)
    
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(content)
    
    print(f"Cleaned content written to {file_path}")


In [9]:
# from chroma_search import get_chroma_hybrid_search

# hyb = get_chroma_hybrid_search("blog_store")
# blogs = get_markdown_files("blogs")
# for blog in blogs:
#     name = blog.split("\\")[-1]
#     clean_md(blog)
#     blog_content = read_file_content(blog)
#     embed(blog, hyb)
#     hyb._load_persisted_data()

Add of existing embedding ID: 0.md_part_0
Insert of existing embedding ID: 0.md_part_0


Data loaded from chroma_store.pkl
Collection initialized: blog_store
Cleaned content written to blogs\0.md
Data persisted to chroma_store.pkl
Document '0.md_part_0' uploaded successfully with ID: 0.md_part_0


Add of existing embedding ID: 0.md_part_1
Insert of existing embedding ID: 0.md_part_1


KeyboardInterrupt: 

In [10]:
topic_list = """
Generate a list of the top 10 underserved blog topics of {topic} with high 
search volume and low competition. Return the results in JSON format with each 
topic as an object containing the topic name and primary keyword only. 
No explanations or additional text should be included in the response. 
Example format: 
{{
  "topics": [
    {{
      "topic": "Topic 1",
      "primary_keyword": "keyword 1"
    }},
    {{
      "topic": "Topic 2",
      "primary_keyword": "keyword 2"
    }}
  ]
}}

 
 - give only valid json
 - do not enclose json in backticks or double quotes
 - markdown formatting is not allowed
"""

In [12]:
import json
def get_underserved_blog_topics(topic):
    global topic_list
    formatted_prompt = topic_list.format(topic=topic)
    response = json.loads(send_request_to_llm(formatted_prompt))
    topic_list = []
    for i in response["topics"]:
        topic_list.append(i["topic"])
    return topic_list


In [ ]:
topic_to_ques = """
Given the following topic:
{topic}

Generate a comprehensive set of three questions that would help determine if a blog thoroughly covers this topic. Consider questions about definitions, implementations, best practices, common challenges, use cases, and comparisons.

Return only a JSON object with an array of questions, without any additional explanations. The output should be in this format:
{{
    "questions": [
        {{"question": "What is {topic}?"}},
        {{"question": "How do you implement {topic}?"}},
        {{"question": "What are the best practices for {topic}?"}},
        ...
    ]
}}

No explanations or additional text should be included in the response.
"""


In [14]:
def get_questions_for_topic_list(topic_list):
    quest_list = []
    for topic in topic_list:
        formatted_prompt = topic_to_ques.format(topic=topic)
        res = json.loads(send_request_to_llm(formatted_prompt))
        for i in res["questions"]:
            quest_list.append(i["question"])
            print(i["question"])
    return quest_list

In [15]:
blog_q_n_a = """ 
## Instruction
You are a helpful assistant that answers questions about blog content. You will receive a question and context from multiple blogs. Your task is to determine if the question can be answered based on the provided context.

If the answer is present in the context:
- Extract the relevant information that answers the question
- Provide the answer in a clear, concise manner

If the answer is not present in the context:
- Return "Not Found"

## Question
{question}

## Context
{context}

## Response Format
Return your response in JSON format without backticks or markdown formatting as follows:

{{
  "question": "[The original question]",
  "answer": "[Your answer or 'Not Found' if not present in context]"
}}

## Guidelines
- Only use information provided in the context to answer the question
- Do not make up information or use external knowledge
- If multiple blog contexts contain relevant information, combine them for a complete answer
- Keep answers concise and relevant
- Maintain the exact JSON format specified above - no backticks, no markdown formatting
- The JSON should be valid and properly formatted
"""

In [16]:
import json
def get_qna_list(quest_list):
    q_n_a_list = []
    
    for question in quest_list:
        context = hyb.search(question, 6, "all")
        print(context)
        context_str = ""
        for j in context:
            context_str += j["document"] + "\n\n"
        print(context_str)
        
        formatted_prompt = blog_q_n_a.format(question=question, context=context_str)
        res = json.loads(send_request_to_llm(formatted_prompt))
        if res["answer"].lower() != "not found":
            q_n_a_list.append(res)
    return q_n_a_list
    

In [ ]:
import json
# q_n_a_list = json.loads(q_n_a_list)
# Write the Q&A list to a JSON file in formatted way
with open("q_n_a_list.json", "w") as file:
    json.dump(q_n_a_list, file, indent=4)

In [ ]:
for qna in q_n_a_list:
    if qna["answer"].lower() != "not found":
        print(qna)

In [97]:
write_blog_prompt = """
Create a high-ranking, engaging, and SEO-optimized blog post based on the information below.

PURPOSE: {purpose}
TARGET AUDIENCE: {target_audience}
PAIN POINTS: {pain_points}
SEARCH INTENT: {search_intent}
TOPIC: {topic}
QUESTIONS & ANSWERS: 
{q_n_a_list}
PRIMARY KEYWORD: {primary_keywords}
SECONDARY KEYWORDS: {secondary_keywords}

Please create a complete blog post with:

1. A catchy headline under 60 characters that includes the primary keyword
2. An engaging introduction with a hook (question/stat/pain point), problem statement, and value proposition
3. A table of contents if the post exceeds 2,000 words
4. Main body content with:
   - Keyword-rich H2 and H3 subheadings
   - Short paragraphs (1-3 sentences)
   - Bullet points and numbered lists
   - Data, statistics, and examples
   - Relevant internal and external links (suggested placeholders)
   - Image placement suggestions
5. Compelling CTAs throughout
6. A conclusion with key takeaways and next steps
7. SEO elements:
   - Meta description (155-160 characters)
   

Use a conversational, engaging, human-like tone with formatting for emphasis. Focus on providing actionable value and use storytelling techniques where appropriate. Make the content scannable, credible, and valuable to the target audience.

"""

In [ ]:
purpose = " Traffic generation, Lead generation"
target_audience = "drupal users"
pain_points = ["Code quality and maintainability", "Development process efficiency","Knowledge transfer and documentation"]

primary_keywords = ["drupal", "drupal seo", "drupal 10 end of life", "best seo tool for drupal", "drupal to wordpress migration", "migrating from drupal​", "drupal migration services", "drupal 7 to 10 migration", "drupal migrate module", "audit drupal", "drupal web development services", "drupal web development company", "drupal 10 module developmen", "drupal website development company​"]
secondary_keywords = ["drupal 9 end of life"]

search_intent = "Transactional, Navigational"
competitors_urls = [
"https://www.herodevs.com/blog-posts/top-3-strategies-for-managing-drupal-7-end-of-life#:~:text=With%20Drupal%207%20EOL%20on%20January%205%2C%202025%2C,...%203%203.%20Prioritize%20Security%20Patches%20and%20Updates",
"https://www.bing.com/search?pglt=417&q=Risk-Free+Drupal+7+Migration+Strategies&cvid=366ef32d8cfb4600ad5eac3a92bbcd68&gs_lcrp=EgRlZGdlKgYIABBFGDkyBggAEEUYOTIGCAEQABhAMgYIAhAAGEAyBggDEAAYQDIGCAQQABhAMgYIBRAAGEAyBggGEAAYQDIGCAcQABhAMgYICBAAGEDSAQgxMzI3ajBqMagCALACAA&FORM=ANNTA1&PC=SMTS",
"https://www.thedroptimes.com/43206/six-migration-options-drupal-7-end-life-in-2025"
]

topic = "Risk-Free Drupal 7 Migration Strategies"

In [ ]:
for i,url in enumerate(competitors_urls):
    webpage_content = fetch_webpage("https://r.jina.ai//"+url)
    #write webpage content to a file
    os.makedirs("blogs", exist_ok=True)
    with open(f"blogs/{i}.md", "w", encoding="utf-8") as file:
        file.write(webpage_content)

    hyb = get_chroma_hybrid_search("blog_store")
    blogs = get_markdown_files("blogs")
    for blog in blogs:
        name = blog.split("\\")[-1]
        clean_md(blog)
        blog_content = read_file_content(blog)
        embed(blog, hyb)
        hyb._load_persisted_data()

In [21]:
topic_list = get_underserved_blog_topics(topic)
questions = get_questions_for_topic_list(topic_list)
qna_list = get_qna_list(questions)

calling LLM 1PWWT1ehinTCU2KNZE7OuE6o8ELkCtRufSrOqOvGxk48lTHZ6ZfCJQQJ99BAACLArgHXJ3w3AAAAACOGsb6z https://ai-tmhub2080459798235.openai.azure.com/ 2024-05-01-preview
<class 'langchain_core.messages.ai.AIMessage'> {
  "topics": [
    {
      "topic": "Best Practices for Drupal 7 Migration",
      "primary_keyword": "Drupal 7 Migration"
    },
    {
      "topic": "Common Pitfalls in Drupal 7 Migration",
      "primary_keyword": "Drupal 7 Migration Pitfalls"
    },
    {
      "topic": "Step-by-Step Guide to Migrating from Drupal 7",
      "primary_keyword": "Step-by-Step Drupal 7 Migration"
    },
    {
      "topic": "Tools for Risk-Free Drupal 7 Migration",
      "primary_keyword": "Drupal 7 Migration Tools"
    },
    {
      "topic": "Case Studies of Successful Drupal 7 Migrations",
      "primary_keyword": "Drupal 7 Migration Case Studies"
    },
    {
      "topic": "Cost-Effective Strategies for Drupal 7 Migration",
      "primary_keyword": "Cost-Effective Drupal 7 Migration"
    }

In [99]:
def write_blog_post(purpose, target_audience, pain_points, search_intent, topic, qna_list, primary_keywords, secondary_keywords):
    formatted_prompt = write_blog_prompt.format(purpose=purpose, target_audience=target_audience, pain_points=pain_points, search_intent=search_intent, topic=topic, q_n_a_list=qna_list, primary_keywords=primary_keywords, secondary_keywords=secondary_keywords)
    response = send_request_to_llm(formatted_prompt)
    return response

blog = write_blog_post(purpose, target_audience, pain_points, search_intent, topic, qna_list, primary_keywords, secondary_keywords)

calling LLM 1PWWT1ehinTCU2KNZE7OuE6o8ELkCtRufSrOqOvGxk48lTHZ6ZfCJQQJ99BAACLArgHXJ3w3AAAAACOGsb6z https://ai-tmhub2080459798235.openai.azure.com/ 2024-05-01-preview
<class 'langchain_core.messages.ai.AIMessage'> # Risk-Free Drupal 7 Migration Strategies You Need to Know

## Introduction

Are you still operating on Drupal 7? If so, you’re not alone. Many organizations rely on this robust content management system, but the clock is ticking. **With Drupal 7 reaching its end of life, the risks associated with security vulnerabilities and performance limitations are mounting.** In fact, continued use of outdated platforms can leave your website exposed and hinder your development process. 

But fear not—there are risk-free strategies for migrating from Drupal 7 to newer versions. In this blog post, we’ll explore actionable techniques, tools, and best practices to ensure a smooth transition without compromising code quality, maintainability, or efficiency. Let’s dive in!

## Table of Contents

In [100]:
with open("blog_post.md", "w", encoding="utf-8") as file:
    file.write(blog)

In [101]:
blog_increse_template = """ 
# Blog Completion Prompt Template

## Instructions
Please complete the following blog article draft by expanding it to approximately 1500 words. Use the provided keywords to guide your content development while maintaining the tone, style, and theme of the original draft.

## Article Draft
{article_draft}

## Keywords
{keywords}

## Requirements
1. Maintain the original voice and writing style
2. Expand the article to approximately 1200 words
3. Incorporate all the provided keywords naturally
4. Ensure logical flow between the existing content and your additions
5. Add appropriate section headings if needed
6. Include a strong conclusion that ties back to the introduction
7. Keep the content informative and engaging

Please complete the blog while preserving any existing formatting, links, or special elements in the markdown.

"""

In [102]:

optimizing_prompt = """
Act as Neil Patel, the renowned SEO and content marketing expert from Ubersuggest. I need you to optimize my blog post to achieve high scores on SEO Review Tools content analysis. Please transform the following blog post while maintaining a natural, human-written feel:

1. KEYWORD OPTIMIZATION:
   - Ensure primary keywords appear in the first 100 words
   - Maintain ideal keyword density (1.5-2.5%) without keyword stuffing
   - Include keywords naturally in headings, especially H2s

2. CONTENT STRUCTURE:
   - Create a compelling H1 headline (65-70 characters) with primary keyword
   - Organize content with proper H1, H2, H3, and H4 hierarchy
   - Keep all paragraphs under 300 characters for readability
   - Ensure 40-60% of sentences are under 20 words for better readability scores
   - Include numbered and bulleted lists where appropriate 

3. ENGAGEMENT ELEMENTS:
   - Add compelling questions throughout to improve engagement metrics
   - Create a strong introduction with a hook, problem statement, and solution promise
   - Include transition phrases between sections
   - Add a compelling call-to-action at the end
   - Incorporate statistics or data points with citations


4. READABILITY IMPROVEMENTS:
   - Simplify complex sentences to achieve 60-70 Flesch Reading Ease score
   - Break up text walls with strategic headings
   - Convert passive voice to active voice where possible
   - Ensure variety in sentence beginnings
   - Use power words and emotional triggers strategically

5. Meta Description:
   - Create a meta description (150-160 characters) with primary keyword
   
Maintain my original ideas and voice while making these improvements. Use Neil's conversational yet authoritative style - approachable, data-driven, and with occasional personal insights.

Here's my blog post:
{blog_draft}

Primary Keywords:
{keywords}


give the output in the following JSON format:
{{
  optimized_blog: "the content of the blog" 
}}

- only give json as output.
- only give the blog output in json. Do not explain the optmization and keywords.
- do not include any explaination.
- do not do any markdown formatting.
- do not wrap the json in backticks.

"""


In [103]:
optimize_blog_prompt = optimizing_prompt.format(blog_draft=blog, keywords=primary_keywords)
optimized_blog = send_request_to_llm(optimize_blog_prompt)

calling LLM 1PWWT1ehinTCU2KNZE7OuE6o8ELkCtRufSrOqOvGxk48lTHZ6ZfCJQQJ99BAACLArgHXJ3w3AAAAACOGsb6z https://ai-tmhub2080459798235.openai.azure.com/ 2024-05-01-preview
<class 'langchain_core.messages.ai.AIMessage'> {
  "optimized_blog": "# Risk-Free Drupal 7 Migration Strategies for a Smooth Transition\n\n## Introduction\n\nAre you still running on Drupal 7? If so, you’re not alone. Many organizations still rely on this robust content management system. However, the clock is ticking. **With Drupal 7 reaching its end of life, risks from security vulnerabilities and performance limitations are mounting.** Continued use of outdated platforms can expose your website to threats and hinder your development process.\n\nBut don’t worry! There are risk-free strategies for migrating from Drupal 7 to newer versions. In this blog post, we’ll explore actionable techniques, tools, and best practices to ensure a smooth transition without compromising code quality, maintainability, or efficiency. Let’s di

In [104]:
optimized_blog_dic = json.loads(optimized_blog)
print(optimized_blog_dic["optimized_blog"])

# Risk-Free Drupal 7 Migration Strategies for a Smooth Transition

## Introduction

Are you still running on Drupal 7? If so, you’re not alone. Many organizations still rely on this robust content management system. However, the clock is ticking. **With Drupal 7 reaching its end of life, risks from security vulnerabilities and performance limitations are mounting.** Continued use of outdated platforms can expose your website to threats and hinder your development process.

But don’t worry! There are risk-free strategies for migrating from Drupal 7 to newer versions. In this blog post, we’ll explore actionable techniques, tools, and best practices to ensure a smooth transition without compromising code quality, maintainability, or efficiency. Let’s dive in!

## Table of Contents

1. **Understanding the Challenges of Drupal 7 Migration**  
   - Security Risks  
   - Compatibility Issues  
2. **Tools for Risk-Free Drupal 7 Migration**  
   - Acquia Migrate Accelerate  
   - Drupal Module 

In [105]:

inc_word_prompt = blog_increse_template.format(article_draft=optimized_blog_dic["optimized_blog"], keywords=primary_keywords)
inc_blog = send_request_to_llm(inc_word_prompt)


calling LLM 1PWWT1ehinTCU2KNZE7OuE6o8ELkCtRufSrOqOvGxk48lTHZ6ZfCJQQJ99BAACLArgHXJ3w3AAAAACOGsb6z https://ai-tmhub2080459798235.openai.azure.com/ 2024-05-01-preview
<class 'langchain_core.messages.ai.AIMessage'> # Risk-Free Drupal 7 Migration Strategies for a Smooth Transition

## Introduction

Are you still running on Drupal 7? If so, you’re not alone. Many organizations still rely on this robust content management system. However, the clock is ticking. **With Drupal 7 reaching its end of life, risks from security vulnerabilities and performance limitations are mounting.** Continued use of outdated platforms can expose your website to threats and hinder your development process.

But don’t worry! There are risk-free strategies for migrating from Drupal 7 to newer versions. In this blog post, we’ll explore actionable techniques, tools, and best practices to ensure a smooth transition without compromising code quality, maintainability, or efficiency. Let’s dive in!

## Table of Contents


In [106]:
with open("blog_post.md", "w", encoding="utf-8") as file:
    file.write(inc_blog)